# Theme 1 — Stakeholder Grid

Ce notebook calcule la matrice Pouvoir/Interet, la strategie d'engagement et un score de priorite.

## 1) Charger les donnees
Place `stakeholders_input.csv` dans le meme dossier que ce notebook avant execution.

In [ ]:
import pandas as pd

df = pd.read_csv("stakeholders_input.csv")

# --- Validate ---
required = {"name","role","type","power","interest","expectation"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"Missing columns: {missing}")

df["power"] = df["power"].astype(int)
df["interest"] = df["interest"].astype(int)

## 2) Calcul des quadrants et strategies

In [ ]:
# --- Quadrant rules (configurable) ---
POWER_T = 3
INTEREST_T = 3

def quadrant(p, i):
    if p >= POWER_T and i >= INTEREST_T:
        return "Key players (Manage closely)"
    if p >= POWER_T and i < INTEREST_T:
        return "Keep satisfied"
    if p < POWER_T and i >= INTEREST_T:
        return "Keep informed"
    return "Monitor"

df["quadrant"] = [quadrant(p,i) for p,i in zip(df["power"], df["interest")]

# --- Strategy mapping ---
strategy_map = {
    "Key players (Manage closely)": "Weekly sync + validate definitions + early risk warning",
    "Keep satisfied": "Short updates + benefits + escalate only if needed",
    "Keep informed": "Demo + newsletter + feedback form",
    "Monitor": "Minimal comms + watch changes"
}
df["strategy"] = df["quadrant"].map(strategy_map)

# --- Priority score (simple heuristic) ---
df["priority_score"] = (0.6 * df["power"] + 0.4 * df["interest"]).round(2)
df = df.sort_values("priority_score", ascending=False)
df.head()

## 3) Export des preuves

In [ ]:
# --- Export proofs ---
df.to_csv("theme1_stakeholder_register.csv", index=False)

with open("theme1_engagement_plan.md", "w", encoding="utf-8") as f:
    f.write("# Engagement plan (Theme 1)\n\n")
    for q in df["quadrant"].unique():
        f.write(f"## {q}\n")
        sub = df[df["quadrant"] == q]
        for _, r in sub.iterrows():
            f.write(f"- **{r['name']}** ({r['role']}) — {r['strategy']}\n")
        f.write("\n")

print("✅ Exports generated: theme1_stakeholder_register.csv, theme1_engagement_plan.md")